In [26]:
import pandas as pd


In [27]:
def get_ops_content_from_logs(filename):
    with open(filename, 'r') as f:
        lines = f.readlines()

    host_id_content_dict = {}
    host_id = None
    content = ""
    content_started = False
    for line in lines:
        if line.startswith("Program host id"):
            # extract host id
            host_id = line.split(" ")[3].rstrip().strip(":")        
        elif line.startswith("{"):
            content_started = True
        elif line.startswith("}"):
            content_started = False
            host_id_content_dict[host_id] = content
            content = ""
        elif content_started:
            content += line
    return host_id_content_dict

In [53]:
import re

def calculate_num_of_cores(corerange: str) -> int:
    # Extract the two (x, y) pairs from the string
    matches = re.findall(r"x=(\d+),y=(\d+)", corerange)
    
    if len(matches) != 2:
        raise ValueError("Invalid input format. Expected two (x, y) pairs.")
    
    # Convert extracted values to integers
    x1, y1 = map(int, matches[0])  # Bottom-left corner
    x2, y2 = map(int, matches[1])  # Top-right corner
    
    # Compute number of cores in the 2D range
    num_of_cores = (x2 - x1 + 1) * (y2 - y1 + 1)
    
    return num_of_cores

In [83]:
# Dispatch time in nanoseconds
def calculate_kernel_groups_dispatch_time(num_of_coreranges, has_rtas, kg_cores=0):
    per_core_times = 4.0 + 19.4 * has_rtas

    return 977.7 * num_of_coreranges + per_core_times * kg_cores

def calculate_rta_dispatch_time(rta_size_with_cores):
    has_rtas = len(rta_size_with_cores) > 0
    rta_dispatch_time = 0
    for rta_size, num_of_cores in rta_size_with_cores:
        num_of_rtas = rta_size / 4
        rta_dispatch_time +=  0.12 * num_of_rtas * num_of_cores

    return 312.6 * has_rtas + rta_dispatch_time
    # return 800 if rta_size / 4 < 36 else 800 + (rta_size / 4 - 36) / 8 * 80

def calculate_sem_cb_dispatch_time(num_of_sem_cbs):
    return 300.0 * num_of_sem_cbs# * 21.0 # TODO: 300 per line


def calculate_kernel_bins_dispatch_time(kb_size_with_cores_list):
    print(kb_size_with_cores_list)
    return sum([num_of_cores * 0.0002 * kb_size for num_of_cores, kb_size in kb_size_with_cores_list])



def calculate_total_dispatch_time(kg_dispatch_time, rta_dispatch_time, sem_cb_dispatch_time, kb_dispatch_time):
    return 1980.0 + kg_dispatch_time + rta_dispatch_time + sem_cb_dispatch_time + kb_dispatch_time

In [84]:
def parse_op_content(content):
    # from the content extract number of kernel groups and number of coreranges
    kernel_group = False
    total_num_coreranges = 0
    total_kg_cores = 0
    total_kgs = 0
    kernel_names = []
    num_of_sem_cb_writes = 0
    kernel_bins_per_corerange = dict()
    rta_size_with_cores = []
    total_kb_size = 0
    has_rtas = False
    dismissed_lines = []
    for line in content.split("\n"):
        if line.startswith("  ") and not line.startswith("    "):
            kernel_group = False
        if "Kernel group targeting cores" in line:
            total_kgs += 1
            kernel_group = True
            total_num_coreranges += line.count("[")
            # get total num of cores in coreranges
            for corerange in re.findall(r"\[(.*?)\]", line):
                total_kg_cores += calculate_num_of_cores(corerange)
        elif kernel_group:
            kernel_names.append(line.split("/")[0])
        elif "Sending unique RTAs" in line:
            has_rtas = True
            # expand size and num of cores from
            # Sending unique RTAs size 24 to core range [(x=3,y=6) - (x=3,y=6)] - 1 cores
            size = int(line.split("RTAs size ")[1].split(" ")[0])
            num_of_cores = int(line.split(" ")[-2])
            # for every extra 
            rta_size_with_cores.append((size, num_of_cores))
            # rta_dispatch_time = calculate_rta_dispatch_time(size, has_rtas)
            # rta_dispatch_times.append(rta_dispatch_time)
            kernel_group = False
        elif "Combined Sem/CB write size" in line:
            num_of_sem_cb_writes += 1
            kernel_group = False
        elif "Sending Kernel binary binary size" in line:
            # extract size and corerange string from
            # Sending Kernel binary binary size 96 to [(x=21,y=24) - (x=21,y=24)]
            
            bytes_size = int(line.split("size ")[1].split(" ")[0])
            corerange_str = line.split("to ")[-1]
            # expand num_of_cores from corerange string [(x=21,y=24) - (x=21,y=24)]
            num_of_cores = calculate_num_of_cores(corerange_str)            
            # add size to corerange
            total_kb_size += bytes_size
            kernel_bins_per_corerange[corerange_str] = kernel_bins_per_corerange.get(corerange_str, 0) + bytes_size
            kernel_group = False
        else:
            kernel_group = False
            dismissed_lines.append(line)
    kb_list = [(calculate_num_of_cores(corerange), bytes_size) for corerange, bytes_size in kernel_bins_per_corerange.items()]
            
    # total_kb_num_of_cores = sum([calculate_num_of_cores(corerange) for corerange in kernel_bins_per_corerange.keys()])
    # return dict with key as names
    return {
        "total_kg_num_coreranges": total_num_coreranges,
        "total_kg_num": total_kgs,
        # "total_kg_cores": total_kg_cores,
        'total_kg_dispatch_time': calculate_kernel_groups_dispatch_time(total_num_coreranges, has_rtas, total_kg_cores),
        "kernel_names": kernel_names,
        "rtas": rta_size_with_cores,
        # "rta_dispatch_times": rta_dispatch_times,
        "total_rta_dispatch_time": calculate_rta_dispatch_time(rta_size_with_cores),
        "num_of_sem_cb_writes": num_of_sem_cb_writes,
        "sem_cb_dispatch_time" : calculate_sem_cb_dispatch_time(num_of_sem_cb_writes),
        "kernel_bins_per_corerange": kernel_bins_per_corerange,
        # "total_kb_num_of_cores": total_kb_num_of_cores,
        "total_kb_size": total_kb_size,
        'kernel_bins_dispatch_time': calculate_kernel_bins_dispatch_time(kb_list)
    }
    

In [85]:
filename = r"/home/divanovic/tt-metal/error_new_dispatch_logging_commit.log"

content_dict = get_ops_content_from_logs(filename)
# parse all op contents:
parsed_ops = {k: parse_op_content(v) for k, v in content_dict.items()}
# for each op add k in dict of parsed_ops values
for k, v in parsed_ops.items():
    v["host_id"] = k

# convert json to dataframe
import json
import pandas as pd
# create dataframe from next struct {col0: host_id: {col1:val, col2:val, col3:val, col4:val,...}}
df = pd.DataFrame.from_dict(parsed_ops, orient='index')

df = df[['host_id', 
         'kernel_names',
         'total_kg_num_coreranges',
         'total_kg_num',
         'total_kg_dispatch_time',
         'rtas',
         'total_rta_dispatch_time',
         'num_of_sem_cb_writes',
         'sem_cb_dispatch_time',
         'kernel_bins_per_corerange',
         'total_kb_size',
         'kernel_bins_dispatch_time'
        ]]

dispatch_time_df = df[['host_id', 'kernel_names', 'total_kg_dispatch_time', 'total_rta_dispatch_time', 'sem_cb_dispatch_time', 'kernel_bins_dispatch_time']]
# add total dispatch time column to dispatch_time_df for columns 'total_kg_dispatch_time', 'total_rta_dispatch_time', 'sem_cb_dispatch_time', 'kernel_bins_dispatch_time'
dispatch_time_df["total_time"] = dispatch_time_df.apply(lambda x: calculate_total_dispatch_time(x['total_kg_dispatch_time'], x['total_rta_dispatch_time'], x['sem_cb_dispatch_time'], x['kernel_bins_dispatch_time']), axis=1)
# set num of digits to 2 for 'total_time' column
dispatch_time_df["total_time"] = dispatch_time_df["total_time"].apply(lambda x: round(x, 2))


[]
[(15, 192), (1, 192)]
[(16, 192)]
[(1, 128), (2, 128), (1, 128), (3, 128), (4, 128), (1, 128)]
[(7, 224), (8, 224), (1, 224)]
[(1, 128)]
[(7, 224), (8, 224), (1, 224)]
[(2, 128), (4, 128), (2, 128), (6, 128), (8, 128), (2, 128)]
[(1, 96), (2, 224), (4, 224), (2, 224), (6, 224), (8, 224), (2, 224)]
[(2, 128), (1, 128), (30, 192), (10, 192)]
[(6, 128), (1, 128), (1, 128), (6, 128), (2, 128)]
[(6, 96), (1, 96), (1, 96), (6, 96), (2, 96)]
[(6, 224), (1, 224), (1, 224), (6, 224), (2, 224)]
[(30, 224), (2, 224)]
[(3, 128)]
[(6, 128), (2, 128)]
[(2, 128), (4, 128), (2, 128), (6, 128), (8, 128), (2, 128)]
[(1, 96), (2, 224), (4, 224), (2, 224), (6, 224), (8, 224), (2, 224)]
[(3, 128), (16, 192)]
[(30, 224), (20, 224)]
[(1, 96), (2, 224), (4, 224), (2, 224), (6, 224), (8, 224), (2, 224)]
[(3, 128), (2, 192), (4, 192), (2, 192), (6, 192), (8, 192), (2, 192)]
[(30, 224), (20, 224)]
[(1, 96), (2, 224), (4, 224), (2, 224), (6, 224), (8, 224), (2, 224)]
[(30, 128), (2, 128)]
[(30, 224), (2, 224)]

/tmp/ipykernel_918/1701897775.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dispatch_time_df["total_time"] = dispatch_time_df.apply(lambda x: calculate_total_dispatch_time(x['total_kg_dispatch_time'], x['total_rta_dispatch_time'], x['sem_cb_dispatch_time'], x['kernel_bins_dispatch_time']), axis=1)
/tmp/ipykernel_918/1701897775.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dispatch_time_df["total_time"] = dispatch_time_df["total_time"].apply(lambda x: round(x, 2))


In [86]:
dispatch_time_df.head(35)

,host_id,kernel_names,total_kg_dispatch_time,total_rta_dispatch_time,sem_cb_dispatch_time,kernel_bins_dispatch_time,total_time
0,0,"[ fabric_erisc_datamover, fabric_erisc_...",7007.7,326.04,0.0,0.0000,9313.74
1531,1531,"[ embeddings_tilize, tilize]",2329.8,324.12,600.0,0.6144,5234.53
1597,1597,"[ embeddings_tilize, tilize]",1352.1,324.12,300.0,0.6144,3956.83
1614,1614,"[ reader_dram, writer_l1]",6147.0,374.52,1800.0,0.3072,10301.83
1637,1637,[ reader_mcast_receiver_unary_sharded_ln_pr...,3307.5,352.08,600.0,0.7168,6240.30
1693,1693,"[ llama_shapes_sharded_writer, llama_sh...",1001.1,316.68,300.0,0.0256,3597.81
1708,1708,[ reader_mcast_receiver_unary_sharded_ln_po...,3307.5,362.64,600.0,0.7168,6250.86
1745,1745,"[ reshard_reader, reshard_reader]",6427.8,450.84,1800.0,0.6144,10659.25
1766,1766,[ reader_bmm_tile_layout_in0_ring_all_gathe...,7428.9,477.48,2100.0,1.0944,11987.47
1804,1804,"[ worker_writer, worker_reader, red...",4917.0,357.96,2700.0,1.6128,9956.57


In [ ]:
# export dispatch_time_df to csv
dispatch_time_df.to_csv('dispatch_time_df_comma_sep.csv')
# export full df to csv
df.to_csv('full_df_comma_sep.csv')
df_coreranges = df[['host_id', 'kernel_names', 'total_kg_num_coreranges', 'total_kg_num']]
non_contiguous_coreranges_df = df_coreranges[df_coreranges['total_kg_num_coreranges'] != df_coreranges['total_kg_num']]
# export non_contiguous_coreranges_df to csv
non_contiguous_coreranges_df.to_csv('non_contiguous_coreranges_df_comma_sep.csv')

In [40]:
df_coreranges = df[['host_id', 'kernel_names', 'total_kg_num_coreranges', 'total_kg_num']]

In [42]:
# print all rows where total_kg_num_coreranges is different than total_kg_num 
non_contiguous_coreranges_df = df_coreranges[df_coreranges['total_kg_num_coreranges'] != df_coreranges['total_kg_num']]
non_contiguous_coreranges_df.head(20)


,host_id,kernel_names,total_kg_num_coreranges,total_kg_num
1531,1531,"[ embeddings_tilize, tilize]",2,1
1614,1614,"[ reader_dram, writer_l1]",6,1
1745,1745,"[ reshard_reader, reshard_reader]",6,1
1766,1766,[ reader_bmm_tile_layout_in0_ring_all_gathe...,7,2
1804,1804,"[ worker_writer, worker_reader, red...",4,2
1826,1826,[ reader_tm_tile_layout_nlp_create_qkv_head...,5,2
1874,1874,"[ rotary_embedding_llama_sharded, rotar...",5,2
1892,1892,"[ writer_update_cache_interleaved_start_id,...",5,2
1934,1934,"[ writer_decode_all, reader_decode_all,...",2,1
1991,1991,[ reader_tm_tile_layout_nlp_concat_heads_de...,2,1


In [43]:
# apply lambda to each row of the dataframe
len(non_contiguous_coreranges_df)

24

In [39]:
dispatch_time_df

,host_id,kernel_names,total_kg_dispatch_time,total_rta_dispatch_time,sem_cb_dispatch_time,kernel_bins_dispatch_time,total_time
0,0,"[ fabric_erisc_datamover, fabric_erisc_...",7007.7,326.04,300.0,0.0000,7635.72
1531,1531,"[ embeddings_tilize, tilize]",1352.1,324.12,342.0,65.2288,2085.43
1597,1597,"[ embeddings_tilize, tilize]",1352.1,324.12,321.0,64.6144,2063.81
1614,1614,"[ reader_dram, writer_l1]",1258.5,374.52,426.0,49.8432,2110.84
1637,1637,[ reader_mcast_receiver_unary_sharded_ln_pr...,3307.5,352.08,342.0,66.1504,4069.71
1693,1693,"[ llama_shapes_sharded_writer, llama_sh...",1001.1,316.68,321.0,4.0256,1644.79
1708,1708,[ reader_mcast_receiver_unary_sharded_ln_po...,3307.5,362.64,342.0,66.1504,4080.27
1745,1745,"[ reshard_reader, reshard_reader]",1539.3,450.84,426.0,99.6864,2517.81
1766,1766,[ reader_bmm_tile_layout_in0_ring_all_gathe...,2540.4,477.48,447.0,107.2000,3574.06
1804,1804,"[ worker_writer, worker_reader, red...",2961.6,357.96,489.0,177.5040,3988.04


In [34]:
# save dispatch_df to csv with '\t' delimiter
dispatch_time_df.to_csv("dispatch_time_df.csv", sep='\t', index=False)
# save df to csv with ',' delimiter
dispatch_time_df.to_csv("dispatch_time_df_comma_sep.csv", index=False)

